# ChronoNet

Here we train the model and record the train and validation accuracy. We used subset of the temple dataset for faster training. we used 500 subjects only for training and validation of the model. The recorded metrics are accuracy, binary cross entropy loss, F1 score, Precision and Recall.

- Train Data: 500 subjects

- Test Data: 276 subjects

In [1]:
import torch
from tqdm import tqdm
import torch.nn as nn

from load_data import read_data_arrays, data_file_names, standardize_data, data_loader
from models import ChronoNet
from utils import cal_accuracy, evaluate_model 


BATCH_SIZE = 128
#device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
DEVICE = torch.device("cpu")
NUM_EPOCHS = 5

print("Reading Data....")
sample_size = 500
data_files = data_file_names(sample_size)
(train_features, val_features, test_features,
 train_labels, val_labels, test_labels, test_lengths) = read_data_arrays(
    data_files)
    
print("Scaling Data....")
train_features, val_features, test_features = standardize_data(
    train_features, val_features, test_features)
    
print("Data Loader....")
train_iter = data_loader(train_features, train_labels, DEVICE, BATCH_SIZE)
val_iter = data_loader(val_features, val_labels, DEVICE, BATCH_SIZE)
test_iter = data_loader(test_features, test_labels, DEVICE, BATCH_SIZE, shuffle=False)
    
print("Training Model....")
n_chans = 19
model=ChronoNet(n_chans)
model.to(DEVICE)
loss_func = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1, NUM_EPOCHS + 1):
    print("Epoch", epoch) 
    loss_sum, n = 0.0, 0
    model.train()
    for t, (x, y) in enumerate(tqdm(train_iter)):
        y_pred = model(x)
        y_pred = y_pred.squeeze()
        loss = loss_func(y_pred, y)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
        optimizer.zero_grad()
    
    val_loss = evaluate_model(model, loss_func, val_iter)
    print("Train loss:", loss_sum / (t+1), ",Train Accuracy: ", 
        cal_accuracy(model, train_iter)[0], ",F1: ", 
        cal_accuracy(model, train_iter)[4], ",Precision: ", 
        cal_accuracy(model, train_iter)[2], ",Recall: ", 
        cal_accuracy(model, train_iter)[3])
    print("Val loss:", val_loss, ", Val Accuracy: ", 
        cal_accuracy(model, val_iter)[0], ",F1: ", 
        cal_accuracy(model, val_iter)[4], ",Precision: ", 
        cal_accuracy(model, val_iter)[2], ",Recall: ", 
        cal_accuracy(model, val_iter)[3])

/Users/mohamedr/opt/anaconda3/envs/xai/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Reading Data....
Scaling Data....
Data Loader....
Training Model....
Epoch 1


100%|█████████████████████████████████████████| 692/692 [04:34<00:00,  2.52it/s]


Train loss: 0.6135080639337529 ,Train Accuracy:  0.7996045197740113 ,F1:  0.7803633571525878 ,Precision:  0.8632490341671918 ,Recall:  0.712
Val loss: 0.6022120368429077 , Val Accuracy:  0.7723050847457628 ,F1:  0.741564387672656 ,Precision:  0.8573080686771639 ,Recall:  0.6533559322033898
Epoch 2


100%|█████████████████████████████████████████| 692/692 [04:21<00:00,  2.65it/s]


...

Train loss: 0.5753913293511881 ,Train Accuracy:  0.8453446327683616 ,F1:  0.8308555469049296 ,Precision:  0.9167416618942431 ,Recall:  0.7596836158192091
Val loss: 0.5960082494335257 , Val Accuracy:  0.792 ,F1:  0.7715221924337206 ,Precision:  0.8557739963654386 ,Recall:  0.7023728813559322
Epoch 4


100%|█████████████████████████████████████████| 692/692 [04:30<00:00,  2.55it/s]


Train loss: 0.570163000353499 ,Train Accuracy:  0.8605762711864406 ,F1:  0.8510484192228297 ,Precision:  0.9134727513021845 ,Recall:  0.7966101694915254
Val loss: 0.595359214972624 , Val Accuracy:  0.7965084745762712 ,F1:  0.7804717498628635 ,Precision:  0.8472409686383485 ,Recall:  0.723457627118644
Epoch 5


100%|█████████████████████████████████████████| 692/692 [04:15<00:00,  2.71it/s]


Train loss: 0.565777277498576 ,Train Accuracy:  0.8552542372881355 ,F1:  0.8381267691063485 ,Precision:  0.9506105601100728 ,Recall:  0.7494463276836159
Val loss: 0.5963385198023412 , Val Accuracy:  0.7821016949152543 ,F1:  0.7513346228239846 ,Precision:  0.8748648648648648 ,Recall:  0.6583728813559322


## Testing the model

In [2]:
from collections import Counter
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

ytrue = []
ypreds = []
with torch.no_grad():
    for x, y in test_iter:
        yhat = model(x)
        yhat = [0 if i<0.5 else 1 for i in yhat]
        ytrue.extend(list(y.numpy()))
        ypreds.extend(yhat)
        
y_final = []
yhat_final = []
for i in range(0, len(ytrue), 118): 
    major_y_final = Counter(ytrue[i: i+118]).most_common(1)[0][0]
    major_yhat_final = Counter(ypreds[i: i+118]).most_common(1)[0][0]
    y_final.append(major_y_final)
    yhat_final.append(major_yhat_final)
    
print("Test Accuracy: ", accuracy_score(y_final, yhat_final))
print("Test Confusion: ",confusion_matrix(y_final, yhat_final))
print("Test F1: ", f1_score(y_final, yhat_final))
print("Test Recall: ", recall_score(y_final, yhat_final))
print("Test Precision: ", precision_score(y_final, yhat_final))

Test Accuracy:  0.8188405797101449
Test Confusion:  [[141   9]
 [ 41  85]]
Test F1:  0.7727272727272727
Test Recall:  0.6746031746031746
Test Precision:  0.9042553191489362


# LSTM

In [5]:
from models import LSTM

learning_rate = 5e-3

model = LSTM(input_size=500, num_channels=19, hidden_units=128)

print("Training Model....")
n_chans = 19
DEVICE = torch.device("cpu")
model.to(DEVICE)
loss_func = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
NUM_EPOCHS = 5
BATCH_SIZE = 128

print("started training")

for epoch in range(1, NUM_EPOCHS + 1):
    print("Epoch", epoch) 
    loss_sum, n = 0.0, 0
    model.train()
    for t, (x, y) in enumerate(tqdm(train_iter)):
        y_pred = model(x)
        y_pred = y_pred.squeeze()
        loss = loss_func(y_pred, y)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
        optimizer.zero_grad()
    
    val_loss = evaluate_model(model, loss_func, test_iter)
    print("Train loss:", loss_sum / (t+1), ",Train Accuracy: ", 
        cal_accuracy(model, train_iter)[0], ",F1: ", 
        cal_accuracy(model, train_iter)[4], ",Precision: ", 
        cal_accuracy(model, train_iter)[2], ",Recall: ", 
        cal_accuracy(model, train_iter)[3])
    print("Val loss:", val_loss, ", Val Accuracy: ", 
        cal_accuracy(model, val_iter)[0], ",F1: ", 
        cal_accuracy(model, val_iter)[4], ",Precision: ", 
        cal_accuracy(model, val_iter)[2], ",Recall: ", 
        cal_accuracy(model, val_iter)[3])

Training Model....
started training
Epoch 1


100%|█████████████████████████████████████████| 692/692 [08:51<00:00,  1.30it/s]


Train loss: 0.661770528279288 ,Train Accuracy:  0.7018870056497175 ,F1:  0.624489389259739 ,Precision:  0.843477257872275 ,Recall:  0.49577401129943505
Val loss: 0.67322963032068 , Val Accuracy:  0.5675593220338983 ,F1:  0.2789803877239586 ,Precision:  0.8386000679578661 ,Recall:  0.16732203389830508
Epoch 2


100%|█████████████████████████████████████████| 692/692 [36:10<00:00,  3.14s/it]


Train loss: 0.6248360861071273 ,Train Accuracy:  0.7378757062146892 ,F1:  0.6679738936279842 ,Precision:  0.9108829729096729 ,Recall:  0.5273446327683616
Val loss: 0.6709002801016265 , Val Accuracy:  0.5743050847457627 ,F1:  0.2901074053137365 ,Precision:  0.8727891156462585 ,Recall:  0.17396610169491525
Epoch 3


...

Train loss: 0.6036012842820558 ,Train Accuracy:  0.7931073446327683 ,F1:  0.7555798803929944 ,Precision:  0.9229991520448764 ,Recall:  0.6395706214689265
Val loss: 0.6608413919514301 , Val Accuracy:  0.602 ,F1:  0.3752461022721226 ,Precision:  0.8721246599060104 ,Recall:  0.2390508474576271
Epoch 4


100%|█████████████████████████████████████████| 692/692 [13:37<00:00,  1.18s/it]


Train loss: 0.5900325086075446 ,Train Accuracy:  0.8101694915254237 ,F1:  0.777824798984342 ,Precision:  0.937575718931327 ,Recall:  0.6645875706214689
Val loss: 0.6607672583823111 , Val Accuracy:  0.604 ,F1:  0.37907940895078135 ,Precision:  0.8774606299212598 ,Recall:  0.2417627118644068
Epoch 5


100%|█████████████████████████████████████████| 692/692 [17:01<00:00,  1.48s/it]


Train loss: 0.5812325871231928 ,Train Accuracy:  0.833683615819209 ,F1:  0.8115485564304462 ,Precision:  0.9361394181066313 ,Recall:  0.716225988700565
Val loss: 0.6539293941329507 , Val Accuracy:  0.6249830508474576 ,F1:  0.44049967126890205 ,Precision:  0.8670117459685447 ,Recall:  0.2952542372881356


In [6]:
ytrue = []
ypreds = []
with torch.no_grad():
    for x, y in test_iter:
        yhat = model(x)
        yhat = [0 if i<0.5 else 1 for i in yhat]
        ytrue.extend(list(y.numpy()))
        ypreds.extend(yhat)

y_final = []
yhat_final = []
for i in range(0, len(ytrue), 118): 
    major_y_final = Counter(ytrue[i: i+118]).most_common(1)[0][0]
    major_yhat_final = Counter(ypreds[i: i+118]).most_common(1)[0][0]
    y_final.append(major_y_final)
    yhat_final.append(major_yhat_final)
    
print("Test Accuracy: ", accuracy_score(y_final, yhat_final))
print("Test Confusion: ",confusion_matrix(y_final, yhat_final))
print("Test F1: ", f1_score(y_final, yhat_final))
print("Test Recall: ", recall_score(y_final, yhat_final))
print("Test Precision: ", precision_score(y_final, yhat_final))

Test Accuracy:  0.6485507246376812
Test Confusion:  [[149   1]
 [ 96  30]]
Test F1:  0.38216560509554137
Test Recall:  0.23809523809523808
Test Precision:  0.967741935483871


# GCN

In [7]:
import math

import torch

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module


class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
        
        self.LSTM = LSTM(input_size=500, num_channels=19, hidden_units=32)
        self.ChronoNet = ChronoNet(19)
        
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        #print(input.shape, self.weight.shape)
        #support = torch.matmul(input, self.weight)
        support = self.ChronoNet(input)
        print(adj.shape, support.shape)
        output = torch.matmul(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'
    
        
import torch.nn as nn
import torch.nn.functional as F
#from pygcn.layers import GraphConvolution


class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)
    

import numpy as np
import scipy.sparse as sp
import torch


def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot


def load_data(path="../cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test


def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx


def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


from tqdm import tqdm
import torch.nn as nn
import torch
from torch.autograd import Variable

class Block(nn.Module):
  def __init__(self,inplace):
    super().__init__()
    self.conv1=nn.Conv1d(in_channels=inplace,out_channels=inplace,kernel_size=2,stride=2,padding=0)
    self.conv2=nn.Conv1d(in_channels=inplace,out_channels=inplace,kernel_size=4,stride=2,padding=1)
    self.conv3=nn.Conv1d(in_channels=inplace,out_channels=inplace,kernel_size=8,stride=2,padding=3)
    self.relu=nn.ReLU()

  def forward(self,x):
    x1=self.relu(self.conv1(x))
    x2=self.relu(self.conv2(x))
    x3=self.relu(self.conv3(x))
    x=torch.cat([x1,x3,x3],dim=1)
    return x

class ChronoNet(nn.Module):
  def __init__(self,channel):
    super().__init__()
    self.block1=Block(channel)
    self.block2=Block(57)
    self.block3=Block(171)
    self.gru1=nn.LSTM(input_size=513,hidden_size=32,batch_first=True)
    self.gru2=nn.LSTM(input_size=32,hidden_size=32,batch_first=True)
    self.gru3=nn.LSTM(input_size=64,hidden_size=32,batch_first=True)
    self.gru4=nn.LSTM(input_size=96,hidden_size=32,batch_first=True)
    self.gru_linear=nn.Linear(62,19)
    self.flatten=nn.Flatten()
    #self.fc1=nn.Linear(32,1)
    self.relu=nn.ReLU()

  def forward(self,x):
    x = x.squeeze()
    x=self.block1(x)
    
    x=self.block2(x)

    x=self.block3(x)

    x=x.permute(0,2,1)
    gru_out1,_=self.gru1(x)
    gru_out2,_=self.gru2(gru_out1)
    gru_out=torch.cat([gru_out1, gru_out2],dim=2)
    gru_out3,_=self.gru3(gru_out)
    gru_out=torch.cat([gru_out1, gru_out2, gru_out3],dim=2)
    print(gru_out.shape)
    linear_out=self.relu(self.gru_linear(gru_out))
    gru_out4,_=self.gru4(linear_out.permute(0,2,1))
    #gru_out4 = gru_out4.permute(0,2,1)
    return gru_out4

In [8]:
#adj.shape, features.shape
features = torch.rand(3, 19, 32)
adj = torch.rand(3, 19, 19)
aa = torch.matmul(adj, features)
aa.shape

torch.Size([3, 19, 32])

In [9]:
import torch
import torch.nn.functional as F
import torch.optim as optim

# Model and optimizer
model = GCN(nfeat=features.shape[2],
            nhid=features.shape[2],
            nclass=2,
            dropout=0.5)
optimizer = optim.Adam(model.parameters(), lr=0.001)

output = model(features, adj)
output.shape

torch.Size([3, 4, 96])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (12x96 and 62x19)